In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 

Using TensorFlow backend.


In [2]:
filename='test1.dcm'
ds = pydicom.dcmread(filename) 
ds.PatientPosition

'PA'

* note I will not use mean and std for normalisation but rather use min max scaling as used in the image data generator 


In [3]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename) 
    if ds.BodyPartExamined != 'CHEST' or ds.PatientPosition not in ['PA', 'AP'] or int(ds.PatientAge) > 110:
            print("Unable to process this file")
            return None
    img = ds.pixel_array
    return img
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_max,img_min,img_size): 
    img=(img-img_min) / (img_max-img_min)
    proc_img=np.resize(img,img_size)
    
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    model=keras.models.load_model(model_path)
    model.load_weights(weight_path)
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    predictions=model.predict(img)
    predictions=[1 if i>thresh else 0 for i in predictions]
    return predictions

In [4]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "./my_model.json"
weight_path = "./xray_class_my_model.best.hdf5"

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
img_max=255
img_min = 0  

my_model = keras.models.load_model(model_path)
thresh = 0.56

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,img_max,img_min,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Load file test1.dcm ...
[0]
Load file test2.dcm ...
[0]
Load file test3.dcm ...
[0]
Load file test4.dcm ...
Unable to process this file
Load file test5.dcm ...
[0]
Load file test6.dcm ...
Unable to process this file
